Shortest Superstring: 


実務的にはシークエンスエラーに対してその補正の際に使用できる．

問題  
点突然変異の場合と同様に，シーケンスエラーの最も一般的なタイプは，リード中の 1 塩基が誤って解釈されることで生じる。


与えられるもの：
長さが同一（最大 50 bp）のリードが最大 1000 本からなる集合が，FASTA 形式で与えられる。これらのリードの一部には 1 塩基の誤りが含まれている。データセット中の各リード s について，次のいずれかが成り立つ：

- s は正しくシーケンスされており，データセット中に少なくとも 2 回出現する（逆相補配列として出現する場合も含む）。

- s は誤っており，データセット中にちょうど 1 回だけ出現し，データセット中のある正しいリード（またはその逆相補配列）とハミング距離 1 であるものがちょうど 1 本だけ存在する。


出力：
すべての修正を "[old read]->[new read]" の形式で列挙せよ。
（各修正は 1 文字の置換でなければならない。また，修正を出力する順序は任意とする。）

In [1]:
def get_rev_comp(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    rev_comp = ''.join(complement[base] for base in reversed(seq))
    return rev_comp

In [2]:
def parse_fasta(lines):
    fasta_dict = {}
    current_key = None
    current_seq = []

    for line in lines:
        if line.startswith(">"):
            if current_key:
                fasta_dict[current_key] = "".join(current_seq)
            current_key = line[1:]
            current_seq = []
        else:
            current_seq.append(line)

    # 最後のレコードの追加
    if current_key:
        fasta_dict[current_key] = "".join(current_seq)

    return fasta_dict


入力された複数のseqを見てその配列がいくつあるかを数える関数.  
defaltdictも使うこともできそう．

In [3]:
def count_same_seqs(fasta_dict):
    seq_count = {}
    for seq in fasta_dict.values():
        rev_comp = get_rev_comp(seq)
        if seq in seq_count:
            seq_count[seq] += 1
        elif rev_comp in seq_count:
            seq_count[rev_comp] += 1
        else:
            seq_count[seq] = 1
    return seq_count

問題条件より，カウントが2以上であれば 正しい配列，1であれば誤った配列とわかる．  
それらを分けてリストに格納する．

In [4]:
def separate_correct_and_incorrect(seq_count):
    correct_seqs = []
    incorrect_seqs = []
    for seq, count in seq_count.items():
        if count >= 2:
            correct_seqs.append(seq)
        else:
            incorrect_seqs.append(seq)
    return correct_seqs, incorrect_seqs

ハミング距離の計算  

find_fixed_seqは，正しい配列のリストと誤った配列**1つ**が入力
問題より，ハミング距離が1であるものが1つ見つかるはずなので，見つかったらそれを指定の形式で返す．

In [5]:
def calculate_hamming_distance(seq1, seq2):
    return sum(base1 != base2 for base1, base2 in zip(seq1, seq2))

def find_fixed_seq(correct_seqs, incorrect):
    for correct in correct_seqs:
        if calculate_hamming_distance(incorrect, correct) == 1:
            return f"{incorrect}->{correct}"
        rev_comp_correct = get_rev_comp(correct)
        if calculate_hamming_distance(incorrect, rev_comp_correct) == 1:
            return f"{incorrect}->{rev_comp_correct}"

In [6]:
def main(fasta_lines):
    fasta_dict = parse_fasta(fasta_lines)
    seq_count = count_same_seqs(fasta_dict)
    correct_seqs, incorrect_seqs = separate_correct_and_incorrect(seq_count)

    results = []
    for incorrect in incorrect_seqs:
        fix = find_fixed_seq(correct_seqs, incorrect)
        if fix:
            results.append(fix)

    return results

In [7]:
sample_fasta = """>Rosalind_52
TCATC
>Rosalind_44
TTCAT
>Rosalind_68
TCATC
>Rosalind_28
TGAAA
>Rosalind_95
GAGGA
>Rosalind_66
TTTCA
>Rosalind_33
ATCAA
>Rosalind_21
TTGAT
>Rosalind_18
TTTCC
"""

In [8]:
main(sample_fasta.splitlines())

['TTCAT->TTGAT', 'GAGGA->GATGA', 'TTTCC->TTTCA']